In [1]:
from mpnn_train import network
import torch.optim as optim
from torch import nn
import torch

# hannas stark..spain germany...switerzland....dcn
# graph net network in action..........matrix in 3d........brzil


100 100 4
torch.Size([100, 100, 4])
MNN(
  (gru): GRUCell(100, 100)
  (APDReadout): GlobalReadout(
    (mlp1): MLP(
      (seq): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): SELU()
        (2): AlphaDropout(p=0.0, inplace=False)
        (3): Linear(in_features=100, out_features=100, bias=True)
        (4): SELU()
        (5): AlphaDropout(p=0.0, inplace=False)
        (6): Linear(in_features=100, out_features=100, bias=True)
        (7): SELU()
        (8): AlphaDropout(p=0.0, inplace=False)
        (9): Linear(in_features=100, out_features=100, bias=True)
        (10): SELU()
        (11): AlphaDropout(p=0.0, inplace=False)
        (12): Linear(in_features=100, out_features=100, bias=True)
        (13): SELU()
        (14): AlphaDropout(p=0.0, inplace=False)
      )
    )
    (mlp2): MLP(
      (seq): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): SELU()
        (2): AlphaDropout(p=0.0, inplace=F

In [2]:
# network
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = torch.nn.KLDivLoss()

In [ ]:
# data[0][1][0].shape

In [93]:
adps[1].shape

(13, 4)

In [130]:
import pickle
data = pickle.load(open('data_1000_train.pkl',"rb")) #{i:{[node_feat,edge_feat],[adp,edge_feat]}}

In [132]:
# network = MNN(constants)
# network = network.cuda()
# outputs = network(node_feat,edge_feat)

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    n_data = len(data)
    
    for i in range(n_data):#this we have to find......
        gh,adps = data[i]
        gh1,adps1 = data[i+1]
        for i in range(len(adps)):
            # adps[i] = torch.tensor(adps[i],dtype=torch.float64)
            # adps1[i] = torch.tensor(adps1[i],dtype=torch.float64)
            print("shapes numpy are ",adps1[i].shape,adps[i].shape)
        

        # get the inputs; data is a list of [inputs, labels]
        node_feat,edge_feat = torch.Tensor(gh[0]).unsqueeze(0).cuda(),torch.Tensor(gh[1]).unsqueeze(0).cuda()
        node_feat1,edge_feat1 = torch.Tensor(gh1[0]).unsqueeze(0).cuda(),torch.Tensor(gh1[1]).unsqueeze(0).cuda()
        node_feat,edge_feat =  torch.cat([node_feat,node_feat1]),torch.cat([edge_feat,edge_feat1])

        for i in range(len(adps)):
            adps[i] = torch.tensor(adps[i],dtype=torch.float64).unsqueeze(0)
            adps1[i] = torch.tensor(adps1[i],dtype=torch.float64).unsqueeze(0)
            print("shapes are ",adps1[i].shape,adps[i].shape)
        
        

        adps_add,adps_conn =  torch.cat([adps[0],adps1[0]]),torch.cat([adps[1],adps1[1]])

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(node_feat,edge_feat)  #now we have 3 outputs 1 for fterm 
        for i,vars in enumerate(outputs):
            outputs[i] = outputs[i].double()
        for i in outputs:
            print(i.dtype)
        print(outputs[0].shape) #adp,edge_conn,term
        label_term = torch.zeros((2,1),dtype=torch.double).cuda()

        
        dp  =torch.flatten(adps_add,start_dim=1).cuda()
        dp_conn  =torch.flatten(adps_conn,start_dim=1).cuda()
        # print("chec ",dp.shape)
        loss_adp = criterion(outputs[0],dp)
        # print(adps[0].dtype,label_term.dtype,adps[1].dtype)
                #break
        loss_conn = criterion(outputs[1],dp_conn)
        loss_term = criterion(outputs[2],label_term)
        loss = loss_adp+loss_conn+loss_term

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print("loss is ",running_loss)
        # break
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0

print('Finished Training')

shapes numpy are  torch.Size([1, 13, 6, 3, 1, 4]) torch.Size([1, 13, 6, 3, 1, 4])
shapes numpy are  torch.Size([1, 13, 4]) torch.Size([1, 13, 4])
shapes are  torch.Size([1, 1, 13, 6, 3, 1, 4]) torch.Size([1, 1, 13, 6, 3, 1, 4])
shapes are  torch.Size([1, 1, 13, 4]) torch.Size([1, 1, 13, 4])
torch.float64
torch.float64
torch.float64
torch.Size([2, 936])
loss is  0.003423399268052517
Finished Training


C:\Users\ASHISH\AppData\Local\Temp/ipykernel_16180/1844603492.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adps[i] = torch.tensor(adps[i],dtype=torch.float64).unsqueeze(0)
C:\Users\ASHISH\AppData\Local\Temp/ipykernel_16180/1844603492.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adps1[i] = torch.tensor(adps1[i],dtype=torch.float64).unsqueeze(0)
C:\Users\ASHISH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\nn\functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL di

In [ ]:
import torch
from collections import namedtuple

In [6]:
hyperparameters = {
        "mlp1_depth"          : 4,
        "mlp1_dropout_p"      : 0.0,
        "mlp1_hidden_dim"     : 100,
        "mlp2_depth"          : 4,
        "mlp2_dropout_p"      : 0.0,
        "mlp2_hidden_dim"     : 100,
        "hidden_node_features": 100,
        "message_passes"      : 3,
        "message_size"        : 100,
        "n_edge_features"     :4,
        "device"              :'cuda'
    }
import json
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
constants = dotdict(hyperparameters)


In [76]:
constants.n_edge_features

4

In [6]:
from DataPrepsF import load_molecule,MolecularGraph,params
path = "data\pre-training\gdb13_1K\Train.smi"
molecule_set = load_molecule(path)
print(molecule_set)
for mol in molecule_set:
    molecule = MolecularGraph(mol)
    print(molecule)
    break




True


In [85]:
from collections import namedtuple
class SummationMPNN(torch.nn.Module):
    """
    Abstract `SummationMPNN` class. Specific models using this class are
    defined in `mpnn.py`; these are MNN, S2V, and GGNN.
    """
    def __init__(self, constants : namedtuple):

        super().__init__()

        self.hidden_node_features = constants.hidden_node_features
        self.edge_features        = constants.n_edge_features
        self.message_size         = constants.message_size
        self.message_passes       = constants.message_passes
        self.constants            = constants

    def forward(self, nodes : torch.Tensor, edges : torch.Tensor) -> None:
        adjacency = torch.sum(edges, dim=3)

        # **note: "idc" == "indices", "nghb{s}" == "neighbour(s)"
        (edge_batch_batch_idc,
         edge_batch_node_idc,
         edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
        #print("sizes are edge_batch_node_idc ",edge_batch_node_idc)

        (node_batch_batch_idc, node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)

        same_batch = node_batch_batch_idc.view(-1, 1) == edge_batch_batch_idc
        same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc

        # element ij of `message_summation_matrix` is 1 if `edge_batch_edges[j]`
        # is connected with `node_batch_nodes[i]`, else 0
        message_summation_matrix = (same_batch * same_node).float()

        edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]

        # pad up the hidden nodes
        hidden_nodes = torch.zeros(nodes.shape[0],
                                   nodes.shape[1],
                                   self.hidden_node_features,
                                   device=self.constants.device)
        hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()
        node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]
        

        for _ in range(self.message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]

            #print("hello ji ",edge_batch_nghbs.shape,edge_batch_nodes.shape,hidden_nodes.shape)

            message_terms = self.message_terms(edge_batch_nodes,
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

            if len(message_terms.size()) == 1:  # if a single graph in batch
                message_terms = message_terms.unsqueeze(0)

            # the summation in eq. 1 of the NMPQC paper happens here
            messages = torch.matmul(message_summation_matrix, message_terms)

            node_batch_nodes = self.update(node_batch_nodes, messages)
            hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :] = node_batch_nodes.clone() #updated the hidden states

        node_mask = adjacency.sum(-1) != 0
        output    = self.readout(hidden_nodes, nodes, node_mask)

        return output


In [86]:
# from graphinvent.gnn.modules import GlobalReadout 
class MNN(SummationMPNN):
    def __init__(self,constants) -> None:
        super().__init__(constants)
        self.constants       = constants
        #print(self.constants.message_size,self.constants.hidden_node_features,4)
        self.message_weights      = torch.Tensor(self.constants.message_size,
                                            self.constants.hidden_node_features,
                                            4)#edge features
        #print(message_weights.shape)
        if self.constants.device == "cuda":
            self.message_weights = self.message_weights.to("cuda", non_blocking=True)
        
    
        self.message_weights = torch.nn.Parameter(self.message_weights)

        self.gru             = torch.nn.GRUCell(
            input_size=self.constants.message_size,
            hidden_size=self.constants.hidden_node_features,
            bias=True
        )
        
        self.APDReadout = GlobalReadout()

        self.reset_parameters()

    def reset_parameters(self) -> None:
        import math
        stdev = 1.0 / math.sqrt(self.message_weights.size(1))
        self.message_weights.data.uniform_(-stdev, stdev)

    def message_terms(self, nodes : torch.Tensor, node_neighbours : torch.Tensor,
                        edges : torch.Tensor) -> torch.Tensor:
        
        edges_view            = edges.view(-1, 1, 1, self.constants.n_edge_features)
        #print("edges ",edges_view.shape)
        weights_for_each_edge = (edges_view * self.message_weights.unsqueeze(0)).sum(3)
        # print("cudas are ",weights_for_each_edge.is_cuda,node_neighbours.is_cuda)
        return torch.matmul(weights_for_each_edge,
                            node_neighbours.unsqueeze(-1)).squeeze()
    #torch.broadcast_to(x, (3, 3))
    def update(self, nodes : torch.Tensor, messages : torch.Tensor) -> torch.Tensor:
        return self.gru(messages, nodes)

    def readout(self, hidden_nodes : torch.Tensor, input_nodes : torch.Tensor,
                node_mask : torch.Tensor) -> torch.Tensor:
        # graph_embeddings = torch.sum(hidden_nodes, dim=1)
        #print("input to network ",hidden_nodes.shape)
        output           = self.APDReadout(hidden_nodes)
        return output



In [83]:
class GlobalReadout(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp1 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlp2 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        
        self.mlp3 = MLP(in_features=(13+1)*constants.message_size,#max nodes+
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=13*6*3*1*4,
                  dropout_p=0.0)
        self.mlp4 = MLP(in_features=(13+1)*constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=13*4,
                  dropout_p=0.0)
        self.mlpt = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=1,
                  dropout_p=0.0)
        # self.final_l = MLP(in_features=2900,#23*100 #(14+14+1)100
        #           hidden_layer_sizes=[500]*1,
        #           out_features=989,
        #           dropout_p=0.0)
          
    def forward(self,features):
        g= torch.sum(features,dim=1)
        g = g.view(g.shape[0],1,100)
        
        #print("feat are ",features.shape)
        fadd1 = self.mlp1(features)
        fconn1 = self.mlp2(features) 
        fadd1g = torch.flatten(torch.cat([fadd1,g],dim=1),start_dim=1)
        fconn1g = torch.flatten(torch.cat([fconn1,g],dim=1),start_dim=1)
        fadd = self.mlp3(fadd1g)#.unsqueeze(dim=1)
        #print(fadd.shape)

        fconn = self.mlp4(fconn1g)#.unsqueeze(dim=1)
        fterm = self.mlpt(g)
        #print("global readout shapes ",fadd.shape,fconn.shape,fterm.shape)
        #cat = torch.cat((fadd.squeeze(dim=0), fconn.squeeze(dim=0), fterm), dim=2)
        cat = [fadd,fconn,fterm.squeeze(1)]
        return cat
        #apd = self.Softmax()....from original code its removed


In [75]:
network = MNN(constants)
network = network.cuda()
outputs = network(node_feat,edge_feat)


torch.Size([1, 936])
torch.Size([1, 52])
torch.Size([1, 1])


In [39]:
outputs.shape

torch.Size([989])

In [84]:

class MLP(torch.nn.Module):
    def __init__(self, in_features : int, hidden_layer_sizes : list, out_features : int,
                 dropout_p : float) -> None:
        super().__init__()

        activation_function = torch.nn.SELU
        # create list of all layer feature sizes
        fs = [in_features, *hidden_layer_sizes, out_features]
        # create list of linear_blocks
        layers = [self._linear_block(in_f, out_f,
                                     activation_function,
                                     dropout_p)
                  for in_f, out_f in zip(fs, fs[1:])]
        # concatenate modules in all sequentials in layers list
        layers = [module for sq in layers for module in sq.children()]

        # add modules to sequential container
        self.seq = torch.nn.Sequential(*layers)

    def _linear_block(self, in_f : int, out_f : int, activation : torch.nn.Module,
                      dropout_p : float) -> torch.nn.Sequential:
        
        # bias must be used in most MLPs in our models to learn from empty graphs
        linear = torch.nn.Linear(in_f, out_f, bias=True)
        torch.nn.init.xavier_uniform_(linear.weight)
        return torch.nn.Sequential(linear, activation(), torch.nn.AlphaDropout(dropout_p))

    def forward(self, layers_input : torch.nn.Sequential) -> torch.nn.Sequential:
        return self.seq(layers_input)


In [47]:
linear = torch.nn.Linear(100,1000, bias=True)


In [50]:
ip = torch.zeros((10,100))

In [51]:
node.shape
edge.shape

(10, 10, 4)

In [52]:
linear(ip).shape

torch.Size([10, 1000])